In [12]:
sys.path.append("../")
adaptive_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if adaptive_root not in sys.path:
    sys.path.append(adaptive_root)

import logging
import os
import sys
from copy import deepcopy

import numpy as np
from cmdstanpy import CmdStanModel
from IPython.display import display
from simulation import aggregator, estimators, fitters, results, simulators, utils

logger = logging.getLogger("cmdstanpy")
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [13]:
exposed_only = estimators.ExposedOnlyEstimator()
difference = estimators.DifferenceEstimator()

model = CmdStanModel(stan_file="../basic_mlm.stan")
posterior_mean_fitter = fitters.StanFitterBasic(model)

posterior_mean = estimators.PosteriorMeanEstimator(posterior_mean_fitter)

In [14]:
simulator1 = deepcopy(utils.CHICK_SIMULATOR)
simulator1.identifier = {"p": "p = constant 0.5"}
simulator1.mu_b = 0

simulator2 = deepcopy(utils.CHICK_SIMULATOR)
simulator2.identifier = {"p": "p = 0.5 to 0.95"}
simulator2.p = np.linspace(0.5, 0.95, utils.CHICK_J)
simulator2.mu_b = 0

In [15]:
sims = []
for simulator in [simulator1, simulator2]:
    for sigma_b in utils.CHICK_SIGMA_B_GRID:
        new_sim = deepcopy(simulator)
        new_sim.sigma_b = sigma_b
        new_sim.identifier = new_sim.identifier | {"sigma_b": sigma_b}
        sims.append(new_sim)

In [ ]:
aggregator = aggregator.SimulationAggregator(sims)
result = aggregator.simulate(
    estimators=[exposed_only, difference, posterior_mean],
    num_reps=100,
    alpha=0.05,
    show_progress=False,
)

Simulators:   0%|          | 0/22 [00:00<?, ?it/s]

Estimators:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/100 [00:00<?, ?it/s]

Repetition:   0%|          | 0/100 [00:00<?, ?it/s]

Repetition:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
result_means = result.means()
display(result_means)

,p,sigma_b,estimator,prop_signif,sample_rmse,pop_rmse,type_s_rate,rank_corr
0,p = 0.5 to 0.95,0.00,Difference,0.359649,0.066090,0.072992,0.000000,0.469234
1,p = 0.5 to 0.95,0.00,Exposed Only,0.684211,0.030947,0.067363,0.000000,0.860452
2,p = 0.5 to 0.95,0.00,Posterior Mean,0.719298,0.029937,0.044271,0.000000,0.812963
3,p = 0.5 to 0.95,0.01,Difference,0.438596,0.061985,0.084945,0.000000,0.677353
4,p = 0.5 to 0.95,0.01,Exposed Only,0.710526,0.036085,0.067858,0.008772,0.808367
...,...,...,...,...,...,...,...,...
61,p = constant 0.5,0.09,Exposed Only,0.605263,0.096187,0.109866,0.052632,0.465879
62,p = constant 0.5,0.09,Posterior Mean,0.561404,0.040481,0.048756,0.000000,0.736368
63,p = constant 0.5,0.10,Difference,0.464912,0.054996,0.081036,0.008772,0.746070
64,p = constant 0.5,0.10,Exposed Only,0.666667,0.111183,0.119922,0.043860,0.408907


In [ ]:
import plotly.express as px

# define dash styles for the two p‐settings
dash_map = {"p = constant 0.5": "solid", "p = 0.5 to 0.95": "dash"}

fig_rmse = px.line(
    result_means,
    x="sigma_b",
    y="sample_rmse",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="RMSE vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "pop_rmse": "RMSE"},
)

# keep legend showing only estimator
fig_rmse.update_layout(legend_title_text="Estimator")
fig_rmse.show()

In [ ]:
fig_type_s = px.line(
    result_means,
    x="sigma_b",
    y="type_s_rate",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Type S error rate vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "type_s_error": "Type S Error Rate"},
)

# keep legend showing only estimator
fig_type_s.update_layout(legend_title_text="Estimator")
fig_type_s.show()

In [ ]:
fig_corr = px.line(
    result_means,
    x="sigma_b",
    y="rank_corr",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Rank Correlation vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "rank_corr": "Rank Correlation"},
)

# keep legend showing only estimator
fig_corr.update_layout(legend_title_text="Estimator")
fig_corr.show()